In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib as mpl

mpl.font_manager.fontManager.addfont('K2D-Light.ttf')
mpl.rc('font', family='K2D', size=12, weight=200)

# Load datset

In [ ]:
src = pd.read_excel('[Raw Data] Customer Behaviors (Responses).xlsx', dtype={'คุณเกิดวันที่เท่าไหร่' : str})
print(src.shape)
src = src[src.columns[1:]]
src.head()

# Data Prep

In [ ]:
from datetime import datetime

df = src.copy()

# rename columns
int_cols = [ c.replace('คุณมีความสนใจในสิ่งเหล่านี้มากน้อยเพียงใด','Interest') for c in df.columns[:30] ]
con_cols = [ c.replace('คุณบริโภคสิ่งเหล่านี้บ่อยขนาดไหน','Consumption') for c in df.columns[30:60] ]
df.columns = int_cols + con_cols + ['interests', 'dob', 'gender']

# make answers categorical
intr_cattype = pd.CategoricalDtype(categories=['ไม่สนใจอย่างมากที่สุด', 'ไม่สนใจอย่างมาก', 'ไม่สนใจ', 
                                              'เฉยๆ', 'สนใจ', 'สนใจอย่างมาก', 'สนใจอย่างมากที่สุด'], ordered=True)
cons_cattype = pd.CategoricalDtype(categories=['แทบไม่ได้บริโภคเลย', 'หลายเดือนครั้ง', 'เดือนละครั้ง', 
                                               'เดือนละ 2-3 ครั้ง', 'อาทิตย์ละครั้ง', 'แทบทุกวัน'], ordered=True)

for col in df.columns[:30]:
    df[col] = df[col].astype(intr_cattype)

for col in df.columns[30:60]:
    df[col] = df[col].astype(cons_cattype)

# gender
df['gender'] = df['gender'].map({'หญิง':'F', 'ชาย':'M', 'ไม่ต้องการระบุ': np.nan})

# process dates
df['year'] = df['dob'].apply(lambda x: int(x.split()[0].split('-')[0]))
df['year'] = df['year'].apply(lambda x: x if x < 2500 else x - 543)
df['month'] = df['dob'].apply(lambda x: int(x.split()[0].split('-')[1]))
df['day'] = df['dob'].apply(lambda x: int(x.split()[0].split('-')[2]))

df['dob'] = pd.to_datetime(df[['year','month','day']])
df['age'] = df['year'].apply(lambda x: datetime.now().year - x if x < 2010 else np.nan)
df['age_group'] = pd.cut(df['age'], bins=[0,28,32,36,99], right=False, labels=['Below 28','28 - 31', '32 - 35','36+'])

df.drop(['year','month','day'], axis=1, inplace=True)
df.head()

# EDA



In [ ]:
intr_scale = ['ไม่สนใจอย่างมากที่สุด', 'ไม่สนใจอย่างมาก', 'ไม่สนใจ', 'เฉยๆ', 'สนใจ', 'สนใจอย่างมาก', 'สนใจอย่างมากที่สุด']
cons_scale = ['แทบไม่ได้บริโภคเลย', 'หลายเดือนครั้ง', 'เดือนละครั้ง', 'เดือนละ 2-3 ครั้ง', 'อาทิตย์ละครั้ง', 'แทบทุกวัน']

In [ ]:
df.head()

## Check & Drop NA

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df.dropna(inplace=True)
df['age'] = df['age'].astype(int)

## Demographics

In [ ]:
f, axes = plt.subplots(1,3, figsize=(14,4), tight_layout=True)

df['gender'].value_counts(dropna=False).plot(kind='bar', title='Gender', ax=axes[0])
df['age'].plot.hist(bins=15, title='Age Distribution', ax=axes[1])
df.groupby(['age_group','gender']).size().unstack().fillna(0).plot(kind='barh', xlabel='Age Group', ylabel='Number of Respondents', 
                                                                   title='Number of Respondents', ax=axes[2]);
plt.savefig('demographic.png',dpi=100)
plt.show()

## Frequency

### Interests

In [ ]:
def process_interest(df):
    result = df.iloc[:,:30].melt(var_name='question', value_name='answer').groupby(['question','answer'], as_index=False).size().pivot('question','answer','size').fillna(0)
    result = result[intr_scale].reindex(df.columns[:30])
    result = result.apply(lambda x: x/x.sum(), axis=1)
    return result

int_overall_df = process_interest(df)
int_male_df = process_interest(df[df['gender']=='M'])
int_female_df = process_interest(df[df['gender']=='F'])

In [ ]:
def plot_freq_heatmap(df, cmap, title):
    ax = sns.heatmap(df, cmap=cmap, annot=True, linewidth=.1, vmin=0, cbar=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=65, ha='right')
    ax.set_title(title)
    return ax

plt.subplots(1,3, figsize=(16,14), tight_layout=True)

plt.subplot(1,3,1)
plot_freq_heatmap(int_overall_df, 'Purples', 'Interests - Overall')

plt.subplot(1,3,2)
ax = plot_freq_heatmap(int_male_df, 'Blues', 'Interests - Male')
ax.set_yticks([])

plt.subplot(1,3,3)
ax = plot_freq_heatmap(int_female_df, 'Reds', 'Interests - Female')
ax.set_yticks([])

plt.savefig('interest_freq.png', dpi=100)
plt.show()

จากการดูด้วยตาเบื้องต้น
- ความสนใจในเหล้าเบียร์, การเล่นเกมส์ ของผู้ชายสูงกว่าผู้หญิงอย่างชัดเจน
- ความสนใจเรื่องแต่งหน้า ของผู้หญิงสูงกว่าผู้ชายอย่างชัดเจน
- ส่วนใหญ่ให้ความสนใจในอาหารญี่ปุ่น อาหารไทย อาหารอีสาน ชาบู ปิ้งย่าง การท่องเที่ยว การดูซีรีส์ ดูหนัง Youtube Netflix

**ข้อสังเกต** อาจจะมี bias ค่อนไปทางบวกหลายเรื่อง future work น่าจะลอง handle เรื่อง bias โดยการคำนวณพวก global bias 


### Consumptions

In [ ]:
def process_consumption(df):
    result = df.iloc[:,30:60].melt(var_name='question', value_name='answer').groupby(['question','answer'], as_index=False).size().pivot('question','answer','size').fillna(0)
    result = result[cons_scale].reindex(df.columns[30:60])
    result = result.apply(lambda x: x/x.sum(), axis=1)
    return result

con_overall_df = process_consumption(df)
con_male_df = process_consumption(df[df['gender']=='M'])
con_female_df = process_consumption(df[df['gender']=='F'])

In [ ]:
plt.subplots(1,3, figsize=(16,14), tight_layout=True)

plt.subplot(1,3,1)
plot_freq_heatmap(con_overall_df, 'Purples', 'Consumptions - Overall')

plt.subplot(1,3,2)
ax = plot_freq_heatmap(con_male_df, 'Blues', 'Consumptions - Male')
ax.set_yticks([])

plt.subplot(1,3,3)
ax = plot_freq_heatmap(con_female_df, 'Reds', 'Consumptions - Female')
ax.set_yticks([])

plt.savefig('consumption_freq.png',dpi=100)
plt.show()


- การบริโภคสินค้าแต่งหน้า, สกินแคร์ ผู้ชายแทบไม่ซื้อเลย
- การเล่นเกม, content เกี่ยวกับเกม ผู้ชายบริโภคสูงกว่าผู้หญิงอย่างชัดเจน ผู้หญิงกลุ่มใหญ่บอกว่าไม่ได้เล่นเลย (แต่รองลงมาเป็นกลุ่มที่ได้เล่นบ่อยมาก)
- ได้ดู youtube บ่อยมากเป็นส่วนใหญ่

## Descriptive & Distributions

### Encode responses

In [ ]:
intr_questions = list(df.columns[:30])
cons_questions = list(df.columns[30:60])

intr_mapper = { v:i for i,v in enumerate(intr_scale,1) }
cons_mapper = { v:i for i,v in enumerate(cons_scale,1) }
print(intr_mapper, cons_mapper, sep='\n')

In [ ]:
score_df = df[list(df.columns[:60]) + ['gender','age_group']].copy()

for col in score_df.columns[:30]:
    score_df[col] = score_df[col].map(intr_mapper).astype(int)

for col in score_df.columns[30:60]:
    score_df[col] = score_df[col].map(cons_mapper).astype(int)

score_df = score_df.melt(id_vars=['gender','age_group'], var_name='question', value_name='score')
print(score_df.shape)
score_df.head()

In [ ]:
intr_score_df = score_df[score_df['question'].isin(intr_questions)]
cons_score_df = score_df[score_df['question'].isin(cons_questions)]

### Descriptive

In [ ]:
with pd.option_context('display.max_rows', None):
    display(score_df.groupby(['question','gender'])['score'].agg([len, np.mean, np.std, np.median, min, max]).reindex( 
        pd.MultiIndex.from_product([df.columns[:60],['F','M']], names=['Question', 'Gender']),
        fill_value=0
    ))

### Side-by-Side Distributions

Interest vs Consumption

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,17), tight_layout=True)

ax = axes[0]
ax.axvline(x=4, color='black',alpha=1, linestyle='--')
ax = sns.boxplot(x='score',y='question', hue='gender',data=intr_score_df, ax=ax,palette="Pastel1")

ax = axes[1]
ax.axvline(x=3.5, color='black',alpha=1, linestyle='--')
ax = sns.boxplot(x='score',y='question', hue='gender',data=cons_score_df, ax=ax,palette="Pastel1")
ax.yaxis.tick_right()
ax.legend().set_visible(False)
plt.savefig('boxplot.png',dpi=100)
plt.show()

- ความสนใจในอาหารจีนพอมี แต่การได้บริโภคนั้นน้อย อาจจะเป็นเพราะไม่ค่อยมีร้านให้บริโภคมากเท่าอาหารไทย หรืออาหารญี่ปุ่น
- ความสนใจในบุฟเฟ่ต์ของผู้หญิงมี spread กว้างแต่เอาเข้าจริงการบริโภคก็ไม่ได้มากนัก
- การเล่นเกมถึงแม้ความสนใจของผู้หญิงจะอยู่กลาง ๆ แต่มีส่วนหนึ่งได้เล่นค่อนข้างบ่อย อาจจะเป็นเกมลักษณะ casual ที่เล่นฆ่าเวลาหรือไม่
- เนื่องจากเป็นนักศึกษาปริญญาโทสาขา BADS ความสนใจและการได้บริโภคของการอ่านหนังสือ บทความออนไลน์ e-Learning จึงค่อนไปทางบวก (as expected)

## Correlations

In [ ]:
score_df = df[list(df.columns[:60]) + ['gender','age']].copy()

for col in score_df.columns[:30]:
    score_df[col] = score_df[col].map(intr_mapper).astype(int)

for col in score_df.columns[30:60]:
    score_df[col] = score_df[col].map(cons_mapper).astype(int)


In [ ]:
intr_df = score_df[list(score_df.columns[:30]) + ['gender','age']]
cons_df = score_df[list(score_df.columns[30:60]) + ['gender','age']]

### Interest Correlation (Overall)

In [ ]:
temp = intr_df.corr().reset_index().melt('index')
# remove self-corr
temp = temp[(temp['value']!=1)&(temp['value']!=-1)]
# remove duplicate pairs
temp = temp.groupby('value', as_index=False).first()
temp.columns = ['Correlation','Interest 1', 'Interest 2']

#### Top Positive Correlations (Overall)

In [ ]:
temp[temp['Correlation']>=0.5].sort_values('Correlation', ascending=False)

- สนใจปิ้งย่างจะสนใจบุฟเฟ่ต์ด้วย
- สนใจ content review ร้านอาหาร จะสนใจ อาหารญี่ปุ่น

#### Top Negative Correlations (Overall)

In [ ]:
temp[temp['Correlation']<=-0.25].sort_values('Correlation')

- ความสนใจการเล่นเกม สัมพันธ์เชิงลบ กับความสนใจแต่งหน้า -> คาแรคเตอร์ ชาย หญิง
- ความสนใจด้านชาบู กับ บุฟเฟ่ต์ เป็นที่นิยามในกลุ่มคนอายุน้อยมากกว่า

#### Correlation Matrix (Overall)

In [ ]:
with sns.plotting_context("notebook", font_scale=.95):
    plt.figure(figsize=(20,15))
    ax = sns.heatmap(intr_df.corr(), cmap='bwr', vmin=-.9, vmax=.9, annot=True, fmt='.2f', cbar_kws={"shrink":.70})
    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=50, fontsize=11, ha='left')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=11)
    plt.title('Interest Correlation - Overall')
    plt.savefig('interest_corr_overall.png', dpi=100)
    plt.show()

### Interest Correlation (Female)

In [ ]:
temp = intr_df[intr_df['gender']=='F'].corr().reset_index().melt('index')
# remove self-corr
temp = temp[(temp['value']!=1)&(temp['value']!=-1)]
# remove duplicate pairs
temp = temp.groupby('value', as_index=False).first()
temp.columns = ['Correlation','Interest 1', 'Interest 2']

#### Top Positive Correlations (Female)

In [ ]:
temp[temp['Correlation']>=0.5].sort_values('Correlation', ascending=False)

**ผู้หญิง**
- (+) สนใจบำรุ่งผิว <-> สนใจแต่งหน้า
- (+) content - review ร้านอาหาร กับ อาหารญี่ปุ่น ชาบู <- content ที่ดูอาจจะเป็น content อาหารญี่ปุ่น  
not done -> TODO

#### Top Negative Correlations (Female)

In [ ]:
temp[temp['Correlation']<=-0.30].sort_values('Correlation')

#### Corelation Matrix (Female)

In [ ]:
with sns.plotting_context("notebook", font_scale=.95):
    plt.figure(figsize=(20,15))
    ax = sns.heatmap(intr_df[intr_df['gender']=='F'].corr(), cmap='bwr', vmin=-.9, vmax=.9, annot=True, fmt='.2f', cbar_kws={"shrink":.70})
    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=50, fontsize=11, ha='left')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=11)
    plt.title('Interest Correlation - Female')
    plt.savefig('interest_corr_female.png', dpi=100)
    plt.show()

### Interest Correlation (Male)

In [ ]:
temp = intr_df[intr_df['gender']=='M'].corr().reset_index().melt('index')
# remove self-corr
temp = temp[(temp['value']!=1)&(temp['value']!=-1)]
# remove duplicate pairs
temp = temp.groupby('value', as_index=False).first()
temp.columns = ['Correlation','Interest 1', 'Interest 2']

#### Top Positive Correlations (Male)

In [ ]:
temp[temp['Correlation']>=0.5].sort_values('Correlation', ascending=False)

#### Top Negative Correlations (Male)

In [ ]:
temp[temp['Correlation']<=-0.30].sort_values('Correlation')

- ผู้ชาย อายุน้อยสนใจ บัฟเฟต์, ชาบูชาบู, ดูซีรีส์, content การเมือง กลับกันถ้าอายุเยอะ
- ผู้ชาย สนใจการเล่นเกม จะไม่สนใจอาหารอีสาน (55555 อะไรเนี่ย)
- ผู้ชาย สนใจเหล้าเบียร์ จะสวนทางกับ สนใจอ่านหนังสือ และอ่านบทความออนไลน์


#### Correlation Matrix (Male)

In [ ]:
with sns.plotting_context("notebook", font_scale=.95):
    plt.figure(figsize=(20,15))
    ax = sns.heatmap(intr_df[intr_df['gender']=='M'].corr(), cmap='bwr', vmin=-.9, vmax=.9, annot=True, fmt='.2f', cbar_kws={"shrink":.70})
    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=50, fontsize=11, ha='left')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=11)
    plt.title('Interest Correlation - Male')
    plt.savefig('interest_corr_male.png', dpi=100)
    plt.show()

- ชาบู บุฟเฟ่ต์ ปิ้งย่าง ชาย อายุน้อย

### Consumption Correlation

In [ ]:
temp = cons_df.corr().reset_index().melt('index')
# remove self-corr
temp = temp[(temp['value']!=1)&(temp['value']!=-1)]
# remove duplicate pairs
temp = temp.groupby('value', as_index=False).first()
temp.columns = ['Correlation','Consumption 1', 'Consumption 2']

#### Top Positive Consumption Correlation

In [ ]:
temp[temp['Correlation']>=0.5].sort_values('Correlation', ascending=False)

#### Top Negative Consumption Correlation

In [ ]:
temp[temp['Correlation']<=-0.30].sort_values('Correlation')

- บริโภคบุฟเฟต์เยอะ แต่อ่านหนังสือน้อย 5555 

#### Correlation Matrix

In [ ]:
with sns.plotting_context("notebook", font_scale=.95):
    plt.figure(figsize=(20,15))
    ax = sns.heatmap(cons_df.corr(), cmap='bwr', vmin=-.9, vmax=.9, annot=True, fmt='.2f', cbar_kws={"shrink":.70})
    ax.xaxis.set_ticks_position('top')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=50, fontsize=11, ha='left')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=11)
    plt.title('Consumption Correlation - Overall')
    plt.savefig('consumption_corr_overall.png', dpi=100)
    plt.show()

In [ ]:
temp = intr_df.corrwith(cons_df.set_axis( intr_df.columns, axis='columns', inplace=False)).reset_index().melt('index')
head = list(cons_df.columns)
head.remove('gender')
temp['variable'] = pd.DataFrame(head)
temp.columns = ['Interest', 'Consumption','Correlation']
temp_desc = temp.sort_values('Correlation', ascending=False)
temp_desc

In [ ]:
temp = temp[temp['Correlation']<=0.50].sort_values('Correlation', ascending=True)
temp

In [ ]:
ixc = temp_desc.pivot("Consumption", "Interest", "Correlation")
plt.figure(figsize=(20,15))
ax = sns.heatmap(ixc, cmap='bwr', vmin=-.9, vmax=.9, annot=True, fmt='.2f', cbar_kws={"shrink":.70})

plt.savefig('InxCo_overall.png',dpi=100)
plt.show()

# K-Means (Unsupervised Clustering)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score



In [ ]:
score_km = df[list(df.columns[:60])].copy()

for col in score_km.columns[:30]:
    score_km[col] = score_km[col].map(intr_mapper).astype(int)

for col in score_km.columns[30:60]:
    score_km[col] = score_km[col].map(cons_mapper).astype(int)

In [ ]:
sil = []
sse = []
n = range(2, 15)
for i in n:
    
  kmeans = KMeans(n_clusters = i).fit(score_km)
  
  sse.append(kmeans.inertia_)
 
  labels = kmeans.labels_
  sil.append(silhouette_score(score_km, labels, metric = 'manhattan'))
 
plt.subplot(1, 2, 1)
plt.plot(n, sse)
 
plt.subplot(1, 2, 2)
plt.plot(n, sil)

plt.savefig('K-Mean.png',dpi=100)
plt.show()